# Key Features

In [1]:
# import useful libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
# importing the data

movies=pd.read_csv('Data/')

graph title length (words, characters)

In [ ]:
# Helper function to count the words in movie titles
def count_words(x):
    # remove eventual spaces at the beginning and at the end of the string, split the string, count the words
    return len(x.strip().split())

In [ ]:
# Plot the join distributions of audience IMdB rating & title length
# As there are more short titles, looking only at the mean rating for each title length is biased

jplot = sns.jointplot(x='Title_length', y='Audience_imdb_rating', data=cmu_imdb, joint_kws={'s' : 25, 'alpha': 0.2})
plt.xlabel('Title length')
plt.ylabel('IMdB audience rating')
jplot.fig.suptitle('Join distributions of audience IMdB rating & title length')

graph part-of-speech : grammatical (verbs, adjective)


graph part-of-speech : proper nouns (characters, dates, location)


graph sentiment analysis 


graph document classification or topic detection